In [ ]:

def picas(n1, n2):
  P = 0
  for i in range(4):
    for j in range(4):
      if(i!=j):
        if(n1[i]==n2[j]):
          P += 1
  return P

def fijas(n1, n2):
  F = 0
  for i in range(4):
    if(n1[i]==n2[i]):
      F += 1
  return F

class Agent:
  def __init__(self):
    self.options = []
    self.last = None

  def ask(self):
    return self.options[len(self.options)//2]

  def init(self):
    self.option = []
    for i in range(10):
      for j in range(10):
        if(j!=i):
          for k in range(10):
            if(k!=i and k!=j):
              for l in range(10):
                if(l!=i and l!=j and l!=k):
                  self.options.append([i, j, k, l])
    return self.ask()

  def compute(self, percept ):
    if(percept == 'iniciar'):
      self.last = self.init()
    else:
      F = percept['fijas']
      if(F==4):
        return "Soy un dios en picas y fijas"
      P = percept['picas']
      opt = []
      for i in range(len(self.options)):
        P2 = picas(self.last, self.options[i])
        F2 = fijas(self.last, self.options[i])
        if(P==P2 and F==F2):
          opt.append(self.options[i])
      self.options = opt
      if(len(self.options)==0):
        return "Usuario tramposo no me diste la información correcta. Humano tenia que ser"
      self.last = self.ask()
    txt = ''
    for i in range(4):
      txt += str(self.last[i])
    return txt

# Programa principal
a = Agent()
a2 = Agent()
n = a.compute('iniciar')
while(True):
  print(n)
  f = int(input('Fijas?'))
  p = int(input('Picas?'))
  n = a.compute({'picas':p, 'fijas':f})

5012


KeyboardInterrupt: Interrupted by user